## MAX CLIQUE PROBLEM

In [1]:
import networkx as nx # крутая штука для операций с графами
import cplex
import time
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from threading import Timer # для остановки алгоритма по достижении лимита по времени
from copy import copy
from sklearn.preprocessing import MinMaxScaler # это я скорее всего немного странную эвристику придумал
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# читаем граф и получаем список ребер и количество вершин
def init_graph(file_name):
    edges = []
    
    with open(file_name, 'r') as file:
        for line in file:
            if line.startswith('c'):  # graph description
                pass
            elif line.startswith('p'):
                p, name, vertices_num, edges_num = line.split()
                pass
            elif line.startswith('e'):
                _, v1, v2 = line.split()
                edges.append((int(v1), int(v2)))
            else:
                continue
    return edges, int(vertices_num)

In [6]:
class MyClass:
    '''
    Здесь добавил какие-то новые переменные, которые не так важны плюс теперь начальные ограничения через рандомные
    расскраски делаются не # вершин раз, а # вершин / 5.
    '''
    def __init__(self, graph, vertices_num, stop_time=30):
        self.save_constraints = []
        self.is_ind_set = True # флаг для получения ind set, ниже напишу как и зачем это надо
        self.bad_ind_set = 0 # смотрим сколько кривых ind sets мы нашли (надо для остановки)
        self.vertices_num = vertices_num # тоже надо для бага с cplex
        self.vertices = [i for i in range(1, vertices_num + 1)]
        Timer(stop_time, self.exitfunc).start() # Установим лимит времени
        self.edges = graph # для проверки что нашли клику
        self.deleted_constraints = [] # чтобы не добавлять уже удаленные ограничения
        self.check_time = False # флажок для остановки ветвления после лимита по времени
        self.nx_graph = nx.Graph(graph) # наш граф
        self.save_solution = None # save solution
        self.nodes = [i for i in range(1, vertices_num + 1)] # список всех вершин
        self.constraints = self.init_constraints(num_random_constraints=int(vertices_num/5)) # инициализируем ограничения
        self.model = self.init_model() # init cplex model
        self.add_constraints(self.constraints) # add constraints
        self.clear_model() # удаляем дублирующие ограничения
        self.model_size = len(self.model.linear_constraints.get_rows()) # сохраняем текущий размер модели
        self.ind_set_index = len(self.model.linear_constraints.get_rows()) # сохраняем индекс independent set
        self.current_maximum_clique = self.heuristic_by_rangdom_coloring_and_degree(num_of_iters=int(vertices_num))
        self.heuristic_clique = self.current_maximum_clique
        self.branch_num = 0 # Номер b&b
        self.model_start_size = len(self.model.linear_constraints.get_rows())
        
        print('START MAX CLIQUE: {}'.format(self.current_maximum_clique))
        print('START BRANCHING') 
        print('MODEL START SIZE: {}'.format(self.model_start_size))
    
    # get graph density, здесь все просто
    def get_graph_density(self):
        max_edges = self.vertices_num * (self.vertices_num - 1) / 2
        current_edges = len(self.edges)
        return current_edges/max_edges
    
    # clear model
    '''
    Чем меньше модель, тем быстрее работает cplex. Давайте удалять излишние ограничения, например если у нас есть ограничение
    [x1, x3] и есть [x1, x2, x3], то [x1, x3] нам в целом не нужно. Сам алгоритм выглядит вот так:
    1) Первый цикл по ограничениям в порядке возрастания;
    2) Второй цикл (внутренний) по ограничениям в порядке убывания (вроде понятно зачем так);
    3) Если нашли большее ограничение, включающее меньшее (одинаковых быть не может), то удаляем меньшее и break
    
    Tips and Tricks (исключительно исходя из моих наблюдений):
    а) Если мы не удалили (не нашли большее) больше чем 30% от первоначального размера ограничений, то break
    б) Раньше стоял early stoppint на len(constraint) > 5, но убрал его, т.к. на самом деле плюс минус не важно   
    '''
    def clear_model(self):
        model_constraints = [i.ind for i in self.model.linear_constraints.get_rows()] # получаем все ограничения
        constraints = sorted(model_constraints, key=lambda x: len(x)) # sort туда
        reversed_constraints = sorted(constraints, key=lambda x: len(x), reverse=True) # sort сюда
        init_len_constraints = len(constraints) # будем делать early stopping после 0.1 * len(constraints), т.к. большие ограничения долго считаются
        not_deleted_constraints = 0
        copy_constraints = copy(constraints)
        remove_from_model_indexes = []
        for constraint in copy_constraints:
            if len(constraint) == 1:
                continue
            not_deleted_constraints += 1
            # эвристическая штука
            if not_deleted_constraints > init_len_constraints * 0.3:
                break
            constraint = set(constraint)
            for check_constraint in reversed_constraints:
                if constraint == set(check_constraint):
                    continue
                elif constraint.issubset(check_constraint):
                    not_deleted_constraints -= 1
                    remove_constraint = sorted(list(constraint))
                    constraint_index = model_constraints.index(remove_constraint)
                    remove_from_model_indexes.append(constraint_index)
                    self.deleted_constraints.append([i + 1 for i in remove_constraint]) # add to deleted_constraints
                    self.save_constraints.remove([i + 1 for i in remove_constraint]) # remove from self.save_constraints                        
                    break  
                    
        # delete constraints from model          
        self.model.linear_constraints.delete(remove_from_model_indexes)

    # ставим флажок для выхода после наступления лимита по времени
    def exitfunc(self):
        self.check_time = True
    
    # инициализируем ограничения
    '''
    По поводу ограничений размера 2, если мы не будем их сразу включать, то у нас есть два варианта когда мы находим
    квазиклику:
    1) Не добавлять нарушенные ограничения размера два, но тогда модель будет работать долго и постоянно находить квазиклики
    2) Добавлять нарушенные ограничения размера два в модель при этом, стараясь их расширить при возможности
    Изначально я пробовал в самом начале добавлять все ограничения размера 2, но clear_model на c-fat работает супер долго,
    т.к. он очень разреженный. В итоге лучший вариант - второй. Однако при этом может быть ситуация когда cplex не находит
    решения, он ошибку тогда выдает, но про это будет ниже.
    '''
    def init_constraints(self, num_random_constraints):
        constraints = []
        
        # color constraints
        strategies = [nx.coloring.strategy_largest_first,
                      nx.coloring.strategy_independent_set,
                      nx.coloring.strategy_connected_sequential_bfs,
                      nx.coloring.strategy_connected_sequential_dfs,
                      nx.coloring.strategy_saturation_largest_first]
        
        for strategy in strategies:
            d = nx.coloring.greedy_color(self.nx_graph, strategy=strategy)
            for color in set(d.values()):
                temp = [key for key, value in d.items() if value == color]
                if len(temp) != 1 and sorted(temp) not in constraints:
                    constraints.append(sorted(temp))
        
        # random color constraints
        for i in range(num_random_constraints):
            d = nx.coloring.greedy_color(self.nx_graph, strategy=nx.coloring.strategy_random_sequential)
            for color in set(d.values()):
                temp = [key for key, value in d.items() if value == color]
                if len(temp) != 1 and sorted(temp) not in constraints:
                    constraints.append(sorted(temp))  
        
        # сохраним ограничения в отдельной переменной
        remove_constraints = []
        for i in constraints:
            if sorted(i) not in self.save_constraints and sorted(i) not in self.deleted_constraints:
                self.save_constraints.append(sorted(i))
            else:
                remove_constraints.append(sorted(i))
                
        for j in remove_constraints:
            constraints.remove(j)
        
        # set constraints - просто ограничения в нужный для cplex api формат
        result_constraints = []
        for constraint in constraints:
            constraint_name = ['x{}'.format(i) for i in constraint]
            result_constraints.append([constraint_name, [1] * len(constraint)])
        return result_constraints
        
    # init model - ничего такого, просто создаем модель и добавляем в нее переменные
    def init_model(self):
        obj = [1] * len(self.nodes)
        upper_bound = [1] * len(self.nodes)
        names = ['x{}'.format(i) for i in self.nodes]
        types = 'C' * len(self.nodes)
        
        model = cplex.Cplex()
        model.set_results_stream(None)        
        model.objective.set_sense(model.objective.sense.maximize)
        model.variables.add(obj=obj, ub=upper_bound,
                              names=names, types=types)
        return model
    
    # добавляем в модель ограничения, тоже все понятно
    def add_constraints(self, constraints):
        constraint_senses = 'L' * len(constraints)
        right_hand_side = [1] * len(constraints)
        constraint_names = ['x{}'.format(i) for i in range(len(constraints))]
        self.model.linear_constraints.add(lin_expr=constraints,
                                          senses=constraint_senses,
                                          rhs=right_hand_side,
                                          names=constraint_names)
    
    '''
    Здесь добавил новую эвристику через поиск независимых множеств на дополненном графе, особого улучшения не дает,
    но пусть будет
    '''
    # Эвристика через рандомную раскраску графа и выбор максимального цвета (рандомно), говорили про нее на лекции
    def heuristic_by_rangdom_coloring_and_degree(self, num_of_iters):   
        max_len = 0
        # random coloring
        for i in range(num_of_iters):
            K = []
            colors = dict(nx.coloring.greedy_color(self.nx_graph, strategy=nx.coloring.strategy_random_sequential))
            while len(colors) != 0 :
                max_color = max(colors.items(), key=lambda x: x[1])[1]
                random_node = random.choice([i[0] for i in colors.items() if i[1] == max_color])
                neigh = list(self.nx_graph.neighbors(random_node))
                K.append(random_node)
                colors.pop(random_node, None)
                colors = {i[0]:i[1] for i in colors.items() if i[0] in neigh}
            if len(K) > max_len:
                max_len = len(K)
                self.save_solution = K
                
        # random degree   
        for i in range(num_of_iters):
            K = []
            nodes = dict(self.nx_graph.degree)
            while len(nodes) != 0:
                max_degree = max(nodes.items(), key=lambda x: x[1])[1]
                random_node = random.choice([i[0] for i in nodes.items() if i[1] == max_degree])
                neigh = list(self.nx_graph.neighbors(random_node))
                K.append(random_node)
                nodes.pop(random_node, None)
                nodes = {i[0]:i[1] for i in nodes.items() if i[0] in neigh}
                if len(K) > max_len:
                    max_len = len(K)
                    self.save_solution = K
        
        # complement graph       
        new_graph = nx.complement(self.nx_graph)
        for i in new_graph.nodes:
            ind_set = nx.maximal_independent_set(new_graph, [i])
            if len(ind_set) > max_len:
                max_len = len(ind_set)
                self.save_solution = ind_set
        
        return max_len
    
    # выбираем переменную для ветвления + проверяем если целочисленное решение
    def get_bb_variable(self, solution):
        bbvar = [(j, i) for j, i in enumerate(solution) if i != 0.0 and not i.is_integer()]
        bbvar = max(bbvar, key = lambda x: x[1], default=(None, None))[0]
        return bbvar
    
    # добавляем ограничение когда ветвимся
    def add_constraint_bb(self, bbvar, constraint_value, current_branch):
        bb_constraint = [[[bbvar], [1.0]]]
        bb_senses = ['E']
        bb_rhs = [constraint_value]
        bb_names = ['branch_{0}'.format(current_branch)]
        self.model.linear_constraints.add(lin_expr=bb_constraint,
                                          senses=bb_senses,
                                          rhs=bb_rhs,
                                          names=bb_names)
        
    # получаем решение системы
    '''
    Здесь есть проблема с тем, что cplex может не находить решение. Например, если мы ветвимся вот так: x1=1, x2=1, x3=1,
    а потом оказывается, что x1 и x2 не имеют ребра и мы добавляем ограничение x1+x2<=1. Тогда cplex ломается и как это
    эффективно по времени предотвратить я не придумал (инициализировать все ограничения 2 - плохо, выше писал почему).
    Как результат, мы просто очень маленькое solution возвращаем в таком случае, пока проблем не было с таким решением
    '''
    def get_solve(self):
        self.model.solve()
        try:
            return self.model.solution.get_values()
        except:
            return [0.00001 for i in range(self.vertices_num)]

    '''
    Здесь просто проверка, что нашли клику. Если нашли квазиклику, то пытаемся расширить ограничения размера 2
    до maximal ind set и добавляем в наши ограничения
    '''
    # проверяем является ли решение кликой
    def is_clique(self, solution):
        new_2_constraints = []
        solution = np.where(np.array(solution) == 1.0)[0] + 1
        iter_count = 0
        for i in solution:
            for j in solution[iter_count:]:
                if i == j:
                    continue
                if (i, j) not in self.edges and (j, i) not in self.edges:
                    new_constraint = nx.maximal_independent_set(self.nx_graph, [i, j])
                    new_2_constraints.append(sorted(new_constraint))
                    self.save_constraints.append(sorted(new_constraint))
            iter_count += 1
        
        if len(new_2_constraints) == 0:
            return True
        else:
            constraints = []
            for constraint in new_2_constraints:
                constraint_name = ['x{}'.format(i) for i in constraint]
                constraints.append([constraint_name, [1] * len(constraint)])
                self.save_constraints.append(sorted(constraint))
            constraint_senses = 'L' * len(new_2_constraints)
            right_hand_side = [1] * len(new_2_constraints)
            constraint_names = []
            for i in new_2_constraints:
                constraint_names.append('x{}'.format(self.ind_set_index))
                self.ind_set_index += 1
            self.model.linear_constraints.add(lin_expr=constraints,
                                                  senses=constraint_senses,
                                                  rhs=right_hand_side,
                                                  names=constraint_names)
            return False

    
    '''
    Здесь эвристика для ind set, про то что я пробовал в следующей ячейке и в целом там вся информация о том,
    почему такая простоя эвристика через цикл по всем вершинам != [0, 1] и поиск с networkx maximal_ind_set
    '''
    # get ind set
    def get_ind_set(self, solution, is_check=False):
                    
        # ЦИКЛ ПО ВСЕМ ВЕРШИНАМ
        first_ind_set = []
        max_weight = -1
        free_vertices = sorted([i for i in self.vertices if solution[i - 1] not in [0, 1]], reverse=True)
        for k in free_vertices:
            ind_set = nx.maximal_independent_set(self.nx_graph, [k]) 
            ind_set_weight = sum([solution[i - 1] for i in ind_set])
            if ind_set_weight > max_weight:
                if sorted(ind_set) not in self.save_constraints and sorted(ind_set) not in self.deleted_constraints:
                    max_weight = ind_set_weight
                    first_ind_set = ind_set
        res_ind_set = first_ind_set
        
        return res_ind_set
    
    # add ind set constraints
    def add_ind_set_constraints(self, ind_set, solution):
        # проверяем есть ли у нас уже такое ограничение
        if sorted(ind_set) in self.save_constraints or sorted(ind_set) in self.deleted_constraints:
            self.bad_ind_set += 1
            return False
        weight_of_in_set = sum([solution[i-1] for i in ind_set])
        if weight_of_in_set < 1.5:
            self.bad_ind_set += 1
            return False
        else:
            self.save_constraints.append(sorted(ind_set))
            # add constraints
            constraints = []
            constraint_name = ['x{}'.format(i) for i in ind_set]
            constraints.append([constraint_name, [1] * len(ind_set)])
            constraint_senses = 'L' * len(constraints)
            right_hand_side = [1] * len(constraints)
            constraint_names = ['x{}'.format(self.ind_set_index)]
            self.model.linear_constraints.add(lin_expr=constraints,
                                                  senses=constraint_senses,
                                                  rhs=right_hand_side,
                                                  names=constraint_names)
            
            self.ind_set_index += 1
            return True
    
    
    # здесь все ветвление
    '''
    Основные моменты по ветвелению:
    1) В самом начале стоит проверка на то, что если модель выросла в 1.5 раза, то отчищаем ее. Как-то поумнее это 
    контролировать не получилось. Раньше еще стояла проверка не то, что если очень много плохих ind set нашли, то перестаем
    их искать, но в итоге перестал ее использовать, т.к. особой разницы нет.
    2) B&C выходит и запускает B&B как только нашли плохой ind set, т.к. его поиск практически никак не рандомизирован и
    смысла искать выходить после n плохих ind set нет.
    3) Плохим ind set считается если он у нас уже был или мы его уже удаляли или его вес < 1.35 или он улучшает solution 
    меньше чем на 1%. Последние две цифры - результат экспериментов.
    
    P.S. Опять же про эксперименты ниже
    '''
    def branch_and_bounds(self):

        # check clear model
        model_size = len(self.model.linear_constraints.get_rows())
        if model_size >= 1.5 * self.model_size:
            self.clear_model()
            self.model_size = len(self.model.linear_constraints.get_rows())
            print('MODEL CLEARED. NEW MODEL SIZE: {}'.format(self.model_size))
        
        if self.check_time != True:
            # get current solution
            solution = self.get_solve()
            solution = [0.0  if i - 1e-9 <= 0.0 else 1.0 if i + 1e-9 >= 1.0 else i for i in solution]
    
            # Если текущее решение больше или равно уже найденной клики + 1.
            if sum(solution) >= self.current_maximum_clique + 1 - 1e-9:   
                
                # проверяем является ли полученное решение (квази)кликой
                bbvar = self.get_bb_variable(solution)
                if bbvar is None:
                    check_solution = self.is_clique(solution)
                    if check_solution:
                        self.current_maximum_clique = sum(solution)
                        self.save_solution = solution
                        print('MAX CLIQUE: {}'.format(self.current_maximum_clique))
                        return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.constraints
                    
                # ЗДЕСЬ НАЧИНАЕТСЯ B&C
                if self.is_ind_set:
                    prev_sum_solution = sum(solution) # сохраняем предыдущее решение чтобы посмотреть когда начинаем ветвиться
                    check_tail = 0
                    while True:
                        ind_set = self.get_ind_set(solution)
                        check_ind_set_constraint = self.add_ind_set_constraints(ind_set, solution)
                        # проверяем что мы добавили новое ограничение
                        if check_ind_set_constraint == False:
                            break
                        # проверяем улучшило ли наше новое ограничение решение
                        solution = self.get_solve()
                        solution = [0.0  if i - 1e-9 <= 0.0 else 1.0 if i + 1e-9 >= 1.0 else i for i in solution] 
                        # проверяем является ли полученное решение (квази)кликой
                        bbvar = self.get_bb_variable(solution)
                        if bbvar is None:
                            check_solution = self.is_clique(solution)
                            if check_solution:
                                self.current_maximum_clique = sum(solution)
                                self.save_solution = solution
                                print('MAX CLIQUE: {}'.format(self.current_maximum_clique))
                                break
                        
                        if sum(solution) >= self.current_maximum_clique + 1 - 1e-9:
                            # проверяем является ли полученное решение (квази)кликой
                            bbvar = self.get_bb_variable(solution)
                            if bbvar is None:
                                check_solution = self.is_clique(solution)
                                if check_solution:
                                    self.current_maximum_clique = sum(solution)
                                    self.save_solution = solution
                                    print('MAX CLIQUE: {}'.format(self.current_maximum_clique))
                                    break
                            
                            if sum(solution) < prev_sum_solution * 0.99:
                                prev_sum_solution = sum(solution)
                            else:
                                self.bad_ind_set += 1
                                break
                        else:
                            break

            
                # ЗДЕСЬ НАЧИНАЕТСЯ B&B   
                if sum(solution) >= self.current_maximum_clique + 1 - 1e-9:
                    bbvar = self.get_bb_variable(solution)
                    # Если мы нашли integer decision
                    if bbvar is None:
                        check_solution = self.is_clique(solution)
                        if check_solution:
                            self.current_maximum_clique = sum(solution)
                            self.save_solution = solution
                            print('MAX CLIQUE: {}'.format(self.current_maximum_clique))
                            return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.constraints
                            
                    # Если решение не целочисленное
                    else:
                        self.branch_num += 1 # увеличиваем ветвление на 1
                        current_branch = self.branch_num
                        self.add_constraint_bb(bbvar, 1.0, current_branch) # добавляем ограничение
                        branch_1 = self.branch_and_bounds() # создаем первую ветку
                        self.model.linear_constraints.delete('branch_{0}'.format(current_branch)) # удаляем ее ограничения
                        self.add_constraint_bb(bbvar, 0.0, current_branch) # добавляем ограничение
                        branch_2 = self.branch_and_bounds() # создаем вторую ветку
                        self.model.linear_constraints.delete('branch_{0}'.format(current_branch)) # удаляем ее ограничения
                        return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.save_constraints, self.bad_ind_set
                else:
                    return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.save_constraints, self.bad_ind_set         
            else:
                return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.save_constraints, self.bad_ind_set         

Большая история о том что я пробовал и что не зашло.
Во-первых по поводу эвристик на поиск независимого множества, вот что я пробовал (не в процессе попыток использования, а по сложности):<br>
1) Цикл по вершинам, не равным [0, 1], который и используется в итоговом решении. Работает быстро и вроде дает результаты неплохие (относительно других, в целом то все плохо). <br>
2) Жадная эвристика: берем вершину с наибольшим весом -> добавляем ее в ind set -> берем вершину с максимальным весом из всех ее не соседей -> добавляем ее в ind set -> берем максимальную вершину из всех не соседей всех вершин из ind set -> добавляем ее в ind set -> ... Работает быстро (какие-то наносекунды) и в целом хорошая эвристика, но хуже первой хоть и не на много. <br>
3) Рандомные свопы (w, 1). Вот здесь много чего пробовал, она в целом была второй эвристикой, которую использовал во время тестов (про первую ниже). Пробовал разное количество итераций, сохранять только что добавленные вершины, рандомизацию только по N самых тяжелых вершин, среднее между весом и длиной через MinMaxScaler + 1 и т.д. Короче много чего, но или я что-то сильно не так делаю или просто вот не работает оно. <br>
4) Первая эвристика, которую я пробовал вот из этой статьи: https://www.researchgate.net/publication/314654518_A_hybrid_iterated_local_search_heuristic_for_the_maximum_weight_independent_set_problem. По идее должно раотать хорошо, да и по времени ее можно свести к той же милисекунде, что и 1) и 2), но к сожалению она оказалась на тестах хуже, чем 1). Может еще есть какие-то интересные эвристики, которые я упустил, но нагуглить каких-то еще хороших статей не нашел. <br>
Еще пробовал раз в N итераций в зависимости от плотности графа или количества вершин выбирать эвристику, но получилось так себе, скорее всего все из-за B&B и его ограничений.

В целом большинство экспериментов было именно с эвристикой, но также что-то делал и в основном цикле B&C. Пробовал разные константы, как-то использовать плотность графа и т.д. Некоторые эксперименты были удачны для определенных графов, например третья эвристика чуть лучше для brock200, но прям плохо для phat. Еще пытался оптимизировать количество вызово модели для получнения решения, сейчас фактически это каждый раз когда мы добавляем ind set и если так не делать, то можем пропустить клику, что плохо, так что остановился на текущем решении, хоть вроде и без излишних запусков модели. Вроде делал что-то еще, но уже забыл.<br>
В итоге получается, что B&C в целом работает хуже, при чем он и ветвится больше раз. Моя ничем не подтвержденная теория заключается в том, что проще изначально реализовать много хороших ограничений с помощью раскрасок или еще как-то, чем потом искать их лишний раз да и в целом стартовать с большего решения (это плохо) и еще дополнительно в начале ветвиться. Как результат, у меня плохо получилось, возможно я упустил что-то очень важное или что-то очень не так реализовал. Графы конечно все еще считаются, но хуже, каких-то ошибок в коде я тоже не нашел и кто виноват: я или сам B&C тоже не знаю.

### СИСТЕМА

##### Intel i5-8300H CPU @2.30GHz
#### RAM 16.0 GB

#### По обозначениям колонок:
- Dimacs Name - имя файла с графом
- Max Clique - реальная максимальная клика
- Time BB - сколько алгоритм работал в секундах
- FInded CLique - найденная клика
- Branches Count - сколько было ветвлений
- Heuristic - начальная эвристика
- Time BB Minutes - Сколько работал в минутах

### Easy Graphs

In [7]:
# EASY GRAPHS
result = []
solutions = []
models, constraints = [], []

dimacs_graphs = [['p_hat300-1.clq', 8], ['C125.9.clq', 34],
                 ['brock200_2.clq', 12], ['keller4.clq', 11]]
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}, FINDING CLIQUE: {}'.format(dimacs_graph[0], dimacs_graph[1]))
    temp = [dimacs_graph[0], dimacs_graph[1]]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    res = MyClass(edges, vertices_num, stop_time=60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
        models.append(res[4])
        constraints.append(res[5])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: p_hat300-1.clq, FINDING CLIQUE: 8
START MAX CLIQUE: 8
START BRANCHING
MODEL START SIZE: 1663
END TIME: 209.58554220199585

DIMACS GRAPH: C125.9.clq, FINDING CLIQUE: 34
START MAX CLIQUE: 32
START BRANCHING
MODEL START SIZE: 467
MAX CLIQUE: 33.0
MAX CLIQUE: 34.0
END TIME: 265.31565737724304

DIMACS GRAPH: brock200_2.clq, FINDING CLIQUE: 12
START MAX CLIQUE: 9
START BRANCHING
MODEL START SIZE: 1446
MAX CLIQUE: 11.0
MAX CLIQUE: 12.0
END TIME: 208.17388272285461

DIMACS GRAPH: keller4.clq, FINDING CLIQUE: 11
START MAX CLIQUE: 11
START BRANCHING
MODEL START SIZE: 764


CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.


END TIME: 96.4209418296814



In [8]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Max Clique', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Max Clique,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,p_hat300-1.clq,8,209,8.0,3363,8,3.48
1,C125.9.clq,34,265,34.0,13969,32,4.42
2,brock200_2.clq,12,208,12.0,4730,9,3.47
3,keller4.clq,11,96,11.0,3409,11,1.60


### brock200

In [6]:
# давайте посмотрим на brock200
result = []
solutions = []

dimacs_graphs = [['brock200_4.clq', 17], ['brock200_3.clq', 15],
                 ['brock200_2.clq', 12], ['brock200_1.clq', 21]]
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}, FINDING CLIQUE: {}'.format(dimacs_graph[0], dimacs_graph[1]))
    temp = [dimacs_graph[0], dimacs_graph[1]]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph[0])
    res = MyClass(edges, vertices_num, stop_time=2*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: brock200_4.clq, FINDING CLIQUE: 17
START MAX CLIQUE: 15
START BRANCHING
MODEL START SIZE: 1823


CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.


MAX CLIQUE: 16.0
MAX CLIQUE: 17.0
END TIME: 1307.1897130012512

DIMACS GRAPH: brock200_3.clq, FINDING CLIQUE: 15
START MAX CLIQUE: 13
START BRANCHING
MODEL START SIZE: 1727


CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.


MAX CLIQUE: 14.0


...

MAX CLIQUE: 20.0


CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.


MAX CLIQUE: 21.0


CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.


END TIME: 5628.17338013649



In [7]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Max Clique', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Max Clique,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,brock200_4.clq,17,1307,17.0,26514,15,21.78
1,brock200_3.clq,15,863,15.0,18595,13,14.38
2,brock200_2.clq,12,265,12.0,6646,10,4.42
3,brock200_1.clq,21,5628,21.0,99337,18,93.80


### c-fat

In [13]:
result = []
solutions = []

dimacs_graphs = ['c-fat200-1.clq', 'c-fat200-2.clq', 'c-fat200-5.clq', 'c-fat500-1.clq', 
                 'c-fat500-10.clq', 'c-fat500-2.clq', 'c-fat500-5.clq']
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph))
    temp = [dimacs_graph]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    res = MyClass(edges, vertices_num, stop_time=2*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: c-fat200-1.clq
START MAX CLIQUE: 12
START BRANCHING
MODEL START SIZE: 622
END TIME: 0.6572251319885254

DIMACS GRAPH: c-fat200-2.clq
START MAX CLIQUE: 24
START BRANCHING
MODEL START SIZE: 1094
END TIME: 0.8349437713623047

DIMACS GRAPH: c-fat200-5.clq
START MAX CLIQUE: 58
START BRANCHING
MODEL START SIZE: 2803
END TIME: 4.733453989028931

DIMACS GRAPH: c-fat500-1.clq
START MAX CLIQUE: 14
START BRANCHING
MODEL START SIZE: 1678
END TIME: 4.429054021835327

DIMACS GRAPH: c-fat500-10.clq
START MAX CLIQUE: 126
START BRANCHING
MODEL START SIZE: 14169
END TIME: 35.404940128326416

DIMACS GRAPH: c-fat500-2.clq
START MAX CLIQUE: 26
START BRANCHING
MODEL START SIZE: 3097
END TIME: 5.711790084838867

DIMACS GRAPH: c-fat500-5.clq
START MAX CLIQUE: 64
START BRANCHING
MODEL START SIZE: 7318
END TIME: 14.458601474761963



In [14]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,c-fat200-1.clq,0,12,0,12,0.00
1,c-fat200-2.clq,0,24,0,24,0.00
2,c-fat200-5.clq,4,58,24,58,0.07
3,c-fat500-1.clq,4,14,0,14,0.07
4,c-fat500-10.clq,35,126,0,126,0.58
5,c-fat500-2.clq,5,26,0,26,0.08
6,c-fat500-5.clq,14,64,0,64,0.23


### san

In [4]:
result = []
solutions = []

dimacs_graphs = ['san200_0.9_3.clq',  'san200_0.7_1.clq', 'san200_0.7_2.clq', 'san200_0.9_1.clq', 'san200_0.9_2.clq',
                 'sanr200_0.7.clq']
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph))
    temp = [dimacs_graph]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    res = MyClass(edges, vertices_num, stop_time=2*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: san200_0.9_3.clq
START MAX CLIQUE: 34
START BRANCHING
MODEL START SIZE: 1612
MAX CLIQUE: 35.0
MAX CLIQUE: 36.0
MAX CLIQUE: 40.0
MAX CLIQUE: 44.0
END TIME: 46.08574843406677

DIMACS GRAPH: san200_0.7_1.clq
START MAX CLIQUE: 22
START BRANCHING
MODEL START SIZE: 1609
MAX CLIQUE: 30.0
END TIME: 3.8838050365448

DIMACS GRAPH: san200_0.7_2.clq
START MAX CLIQUE: 14
START BRANCHING
MODEL START SIZE: 1131


CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.


MAX CLIQUE: 15.0


CPLEX Error  1217: No solution exists.


MAX CLIQUE: 16.0
STOP FININDG IND SETS
MODEL CLEARED. NEW MODEL SIZE: 1247
MAX CLIQUE: 18.0
END TIME: 53.774847745895386

DIMACS GRAPH: san200_0.9_1.clq
START MAX CLIQUE: 51
START BRANCHING
MODEL START SIZE: 1498
MAX CLIQUE: 70.0
END TIME: 17.793482303619385

DIMACS GRAPH: san200_0.9_2.clq
START MAX CLIQUE: 44
START BRANCHING
MODEL START SIZE: 1642
MAX CLIQUE: 60.0
END TIME: 14.739104747772217

DIMACS GRAPH: sanr200_0.7.clq
START MAX CLIQUE: 16
START BRANCHING
MODEL START SIZE: 1987


CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.


MAX CLIQUE: 17.0
STOP FININDG IND SETS
MODEL CLEARED. NEW MODEL SIZE: 2093
MAX CLIQUE: 18.0
END TIME: 5810.843228816986



In [5]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,san200_0.9_3.clq,46,44.0,530,34,0.77
1,san200_0.7_1.clq,3,30.0,0,22,0.05
2,san200_0.7_2.clq,53,18.0,1243,14,0.88
3,san200_0.9_1.clq,17,70.0,0,51,0.28
4,san200_0.9_2.clq,14,60.0,0,44,0.23
5,sanr200_0.7.clq,5810,18.0,144578,16,96.83


### Ну и остальное до кучи

In [4]:
result = []
solutions = []

dimacs_graphs = ['MANN_a9.clq', 'MANN_a27.clq', 'MANN_a45.clq', 'hamming6-2.clq', 'hamming6-4.clq']
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph))
    temp = [dimacs_graph]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    res = MyClass(edges, vertices_num, stop_time=4*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: MANN_a9.clq
START MAX CLIQUE: 16
START BRANCHING
MODEL START SIZE: 65
END TIME: 0.2083284854888916

DIMACS GRAPH: MANN_a27.clq
START MAX CLIQUE: 125
START BRANCHING
MODEL START SIZE: 639
MAX CLIQUE: 126.0
END TIME: 318.0978808403015

DIMACS GRAPH: MANN_a45.clq
START MAX CLIQUE: 341
START BRANCHING
MODEL START SIZE: 1730
MAX CLIQUE: 343.0
MAX CLIQUE: 344.0
END TIME: 19437.888439178467

DIMACS GRAPH: hamming6-2.clq
START MAX CLIQUE: 32
START BRANCHING
MODEL START SIZE: 176
END TIME: 0.1832590103149414

DIMACS GRAPH: hamming6-4.clq
START MAX CLIQUE: 4
START BRANCHING
MODEL START SIZE: 122
END TIME: 0.3354380130767822



In [5]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,MANN_a9.clq,0,16.0,6,16,0.00
1,MANN_a27.clq,318,126.0,2895,125,5.30
2,MANN_a45.clq,19437,344.0,107,341,323.95
3,hamming6-2.clq,0,32.0,0,32,0.00
4,hamming6-4.clq,0,4.0,20,4,0.00


In [8]:
result = []
solutions = []

dimacs_graphs = ['gen200_p0.9_44.clq', 'gen200_p0.9_55.clq', 'p_hat300-2.clq', 'p_hat300-3.clq']
for dimacs_graph in dimacs_graphs:
    print('DIMACS GRAPH: {}'.format(dimacs_graph))
    temp = [dimacs_graph]
    start_time = time.time()
    edges, vertices_num = init_graph('dimacs graphs/' + dimacs_graph)
    res = MyClass(edges, vertices_num, stop_time=4*60*60).branch_and_bounds()
    end_time = time.time() - start_time
    temp.append(end_time)
    try:
        temp.append(res[0])
        temp.append(res[1])
        temp.append(res[2])
        solutions.append(res[3])
    except:
        temp.append('-')
        temp.append('-')
        temp.append('-')
    result.append(temp)
    print('END TIME: {}'.format(end_time))
    print()

DIMACS GRAPH: gen200_p0.9_44.clq
START MAX CLIQUE: 35
START BRANCHING
MODEL START SIZE: 1159


CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.


MAX CLIQUE: 36.0
MAX CLIQUE: 37.0
MAX CLIQUE: 38.0
MAX CLIQUE: 39.0
MAX CLIQUE: 40.0
MAX CLIQUE: 41.0
MAX CLIQUE: 44.0
END TIME: 3645.325154066086

DIMACS GRAPH: gen200_p0.9_55.clq
START MAX CLIQUE: 41
START BRANCHING
MODEL START SIZE: 1167
MAX CLIQUE: 55.0
END TIME: 10.33168888092041

DIMACS GRAPH: p_hat300-2.clq
START MAX CLIQUE: 23
START BRANCHING
MODEL START SIZE: 2680
MAX CLIQUE: 25.0
END TIME: 2406.1895582675934

DIMACS GRAPH: p_hat300-3.clq
START MAX CLIQUE: 31
START BRANCHING
MODEL START SIZE: 3660
MAX CLIQUE: 32.0
MAX CLIQUE: 35.0
MAX CLIQUE: 36.0
END TIME: 14400.22091794014



In [9]:
result_df = pd.DataFrame(result, columns=['Dimacs Name', 'Time BB', 'Finded Clique',
                                          'Branches Count', 'Heuristic'])
result_df['Time BB'] = result_df['Time BB'].astype('int')
result_df['Time BB Minutes'] = round(result_df['Time BB'] / 60, 2)
result_df

,Dimacs Name,Time BB,Finded Clique,Branches Count,Heuristic,Time BB Minutes
0,gen200_p0.9_44.clq,3645,44.0,102264,35,60.75
1,gen200_p0.9_55.clq,10,55.0,1,41,0.17
2,p_hat300-2.clq,2406,25.0,17102,23,40.10
3,p_hat300-3.clq,14400,36.0,118749,31,240.00


In [8]:
# ЭТО БОЛЬШЕ ДЛЯ МЕНЯ КОД, НЕ НАДО ЕГО СМОТРЕТЬ
class MyClass:
    '''
    Здесь добавил какие-то новые переменные, которые не так важны плюс теперь начальные ограничения через рандомные
    расскраски делаются не # вершин раз, а # вершин / 5
    '''
    def __init__(self, graph, vertices_num, stop_time=30):
        self.choose_ind_set_heuristic = 'second' # heuristic type
        self.is_ind_set = True # флаг для получения ind set, ниже напишу как и зачем это надо
        self.bad_ind_set = 0 # смотрим сколько кривых ind sets мы нашли (надо для остановки)
        self.vertices_num = vertices_num # тоже надо для бага с cplex
        self.vertices = [i for i in range(1, vertices_num + 1)]
        Timer(stop_time, self.exitfunc).start() # Установим лимит времени
        self.edges = graph # для проверки что нашли клику
        self.graph_density = self.get_graph_density()
        self.deleted_constraints = [] # чтобы не добавлять уже удаленные ограничения
        self.check_time = False # флажок для остановки ветвления после лимита по времени
        self.nx_graph = nx.Graph(graph) # наш граф
        self.save_solution = None # save solution
        self.nodes = [i for i in range(1, vertices_num + 1)] # список всех вершин
        self.constraints = self.init_constraints(num_random_constraints=int(vertices_num/5)) # инициализируем ограничения
        self.model = self.init_model() # init cplex model
        self.add_constraints(self.constraints) # add constraints
        self.clear_model() # удаляем дублирующие ограничения
        self.model_size = len(self.model.linear_constraints.get_rows()) # сохраняем текущий размер модели
        self.ind_set_index = len(self.model.linear_constraints.get_rows()) # сохраняем индекс independent set
        self.current_maximum_clique = self.heuristic_by_rangdom_coloring_and_degree(num_of_iters=int(vertices_num))
        self.heuristic_clique = self.current_maximum_clique
        self.branch_num = 0 # Номер b&b
        self.model_start_size = len(self.model.linear_constraints.get_rows())
        self.added_ind_set = []
        
        print('START MAX CLIQUE: {}'.format(self.current_maximum_clique))
        print('START BRANCHING') 
        print('MODEL START SIZE: {}'.format(self.model_start_size))
    
    # get graph density
    def get_graph_density(self):
        max_edges = self.vertices_num * (self.vertices_num - 1) / 2
        current_edges = len(self.edges)
        return current_edges/max_edges
    
    '''
    Чем меньше модель, тем быстрее работает cplex. Давайте удалять излишние ограничения, например если у нас есть ограничение
    [x1, x3] и есть [x1, x2, x3], то [x1, x3] нам в целом не нужно. Сам алгоритм выглядит вот так:
    1) Первый цикл по ограничениям в порядке возрастания;
    2) Второй цикл (внутренний) по ограничениям в порядке убывания (вроде понятно зачем так);
    3) Если нашли большее ограничение, включающее меньшее (одинаковых быть не может), то удаляем меньшее и break
    
    Tips and Tricks (исключительно исходя из моих наблюдений):
    а) Если мы не удалили (не нашли большее) больше чем 10% от первоначального размера ограничений, то break
    б) Если длина очередного ограничения больше 5, то break    
    '''
    # clear model
    def clear_model(self):
        model_constraints = [i.ind for i in self.model.linear_constraints.get_rows()] # получаем все ограничения
        constraints = sorted(model_constraints, key=lambda x: len(x)) # sort туда
        reversed_constraints = sorted(constraints, key=lambda x: len(x), reverse=True) # sort сюда
        init_len_constraints = len(constraints) # будем делать early stopping после 0.1 * len(constraints), т.к. большие ограничения долго считаются
        not_deleted_constraints = 0
        copy_constraints = copy(constraints)
        remove_from_model_indexes = []
        for constraint in copy_constraints:
            if len(constraint) == 1:
                continue
            not_deleted_constraints += 1
            # эвристическая штука
            if not_deleted_constraints > init_len_constraints * 0.1:
                break
            constraint = set(constraint)
            for check_constraint in reversed_constraints:
                if constraint == set(check_constraint):
                    continue
                elif constraint.issubset(check_constraint):
                    not_deleted_constraints -= 1
                    remove_constraint = sorted(list(constraint))
                    constraint_index = model_constraints.index(remove_constraint)
                    remove_from_model_indexes.append(constraint_index)
                    self.deleted_constraints.append([i + 1 for i in remove_constraint]) # add to deleted_constraints
                    self.save_constraints.remove([i + 1 for i in remove_constraint]) # remove from self.save_constraints                        
                    break  
                    
        # delete constraints from model          
        self.model.linear_constraints.delete(remove_from_model_indexes)

    # ставим флажок для выхода после наступления лимита по времени
    def exitfunc(self):
        self.check_time = True
    
    '''
    По поводу ограничений размера 2, если мы не будем их сразу включать, то у нас есть два варианта когда мы находим
    квазиклику:
    1) Не добавлять нарушенные ограничения размера два, но тогда модель будет работать долго и постоянно находить квазиклики
    2) Добавлять нарушенные ограничения размера два в модель
    Если бы у меня не было первоначально отчистки модели, я бы выбрал второй вариант, но т.к. отчистка модели скорее всего
    или просто удалит все ограничения размера два или оставит только нужные, которые позже будут в 2) добавлены, то
    разницы по времени работы нет, так что я оставил ограничения размера два
    
    ПЕРЕПИСАТЬ ПОСЛЕ ТЕСТОВ
    
    '''
    # инициализируем ограничения
    def init_constraints(self, num_random_constraints):
        constraints = []
        
        # color constraints
        strategies = [nx.coloring.strategy_largest_first,
                      nx.coloring.strategy_independent_set,
                      nx.coloring.strategy_connected_sequential_bfs,
                      nx.coloring.strategy_connected_sequential_dfs,
                      nx.coloring.strategy_saturation_largest_first]
        
        for strategy in strategies:
            d = nx.coloring.greedy_color(self.nx_graph, strategy=strategy)
            for color in set(d.values()):
                temp = [key for key, value in d.items() if value == color]
                if len(temp) != 1 and sorted(temp) not in constraints:
                    constraints.append(sorted(temp))
        
        # random color constraints
        for i in range(num_random_constraints):
            d = nx.coloring.greedy_color(self.nx_graph, strategy=nx.coloring.strategy_random_sequential)
            for color in set(d.values()):
                temp = [key for key, value in d.items() if value == color]
                if len(temp) != 1 and sorted(temp) not in constraints:
                    constraints.append(sorted(temp))  
        
        # сохраним ограничения в отдельной переменной
        self.save_constraints = constraints
        
        # set constraints - просто ограничения в нужный для cplex api формат
        result_constraints = []
        for constraint in constraints:
            constraint_name = ['x{}'.format(i) for i in constraint]
            result_constraints.append([constraint_name, [1] * len(constraint)])
        return result_constraints
        
    # init model - ничего такого, просто создаем модель и добавляем в нее переменные
    def init_model(self):
        obj = [1] * len(self.nodes)
        upper_bound = [1] * len(self.nodes)
        names = ['x{}'.format(i) for i in self.nodes]
        types = 'C' * len(self.nodes)
        
        model = cplex.Cplex()
        model.set_results_stream(None)        
        model.objective.set_sense(model.objective.sense.maximize)
        model.variables.add(obj=obj, ub=upper_bound,
                              names=names, types=types)
        return model
    
    # добавляем в модель ограничения, тоже все понятно
    def add_constraints(self, constraints):
        constraint_senses = 'L' * len(constraints)
        right_hand_side = [1] * len(constraints)
        constraint_names = ['x{}'.format(i) for i in range(len(constraints))]
        self.model.linear_constraints.add(lin_expr=constraints,
                                          senses=constraint_senses,
                                          rhs=right_hand_side,
                                          names=constraint_names)
    
    # Эвристика через рандомную раскраску графа и выбор максимального цвета (рандомно), говорили про нее на лекции
    def heuristic_by_rangdom_coloring_and_degree(self, num_of_iters):   
        max_len = 0
        # random coloring
        for i in range(num_of_iters):
            K = []
            colors = dict(nx.coloring.greedy_color(self.nx_graph, strategy=nx.coloring.strategy_random_sequential))
            while len(colors) != 0 :
                max_color = max(colors.items(), key=lambda x: x[1])[1]
                random_node = random.choice([i[0] for i in colors.items() if i[1] == max_color])
                neigh = list(self.nx_graph.neighbors(random_node))
                K.append(random_node)
                colors.pop(random_node, None)
                colors = {i[0]:i[1] for i in colors.items() if i[0] in neigh}
            if len(K) > max_len:
                max_len = len(K)
                self.save_solution = K
                
        # random degree   
        for i in range(num_of_iters):
            K = []
            nodes = dict(self.nx_graph.degree)
            while len(nodes) != 0:
                max_degree = max(nodes.items(), key=lambda x: x[1])[1]
                random_node = random.choice([i[0] for i in nodes.items() if i[1] == max_degree])
                neigh = list(self.nx_graph.neighbors(random_node))
                K.append(random_node)
                nodes.pop(random_node, None)
                nodes = {i[0]:i[1] for i in nodes.items() if i[0] in neigh}
                if len(K) > max_len:
                    max_len = len(K)
                    self.save_solution = K
        return max_len
    
    # выбираем переменную для ветвления + проверяем если целочисленное решение
    def get_bb_variable(self, solution):
        bbvar = [(j, i) for j, i in enumerate(solution) if i != 0.0 and not i.is_integer()]
        bbvar = max(bbvar, key = lambda x: x[1], default=(None, None))[0]
        return bbvar
    
    # добавляем ограничение когда ветвимся
    def add_constraint_bb(self, bbvar, constraint_value, current_branch):
        bb_constraint = [[[bbvar], [1.0]]]
        bb_senses = ['E']
        bb_rhs = [constraint_value]
        bb_names = ['branch_{0}'.format(current_branch)]
        self.model.linear_constraints.add(lin_expr=bb_constraint,
                                          senses=bb_senses,
                                          rhs=bb_rhs,
                                          names=bb_names)
        
    # получаем решение системы
    def get_solve(self):
        self.model.solve()
        try:
            return self.model.solution.get_values()
        except:
            return [0.00001 for i in range(self.vertices_num)]

    '''
    Чуть выще уже писал про ограничения размера два и в целом этот код не нужен и дальше не используется, но пусть лежит
    '''
    # проверяем является ли решение кликой
    def is_clique(self, solution):
        new_2_constraints = []
        solution = np.where(np.array(solution) == 1.0)[0] + 1
        iter_count = 0
        for i in solution:
            for j in solution[iter_count:]:
                if i == j:
                    continue
                if (i, j) not in self.edges and (j, i) not in self.edges:
                    new_2_constraints.append(sorted([i, j]))
                    self.save_constraints.append(sorted([i, j]))
            iter_count += 1
        
        if len(new_2_constraints) == 0:
            return True
        else:
            constraints = []
            for constraint in new_2_constraints:
                constraint_name = ['x{}'.format(i) for i in constraint]
                constraints.append([constraint_name, [1] * len(constraint)])
                self.save_constraints.append(sorted(constraint))
            constraint_senses = 'L' * len(new_2_constraints)
            right_hand_side = [1] * len(new_2_constraints)
            constraint_names = []
            for i in new_2_constraints:
                constraint_names.append('x{}'.format(self.ind_set_index))
                self.ind_set_index += 1
            self.model.linear_constraints.add(lin_expr=constraints,
                                                  senses=constraint_senses,
                                                  rhs=right_hand_side,
                                                  names=constraint_names)
            return False

    
    '''
    Здесь эвристика для ind set, ниже расскажу что я пробовал. В целом мы хотим получить ind set с наибольшим весом и размера
    тоже побольше, но в основном ориентируемся на вес.
    1) Жадная эвристика: берем вершину с наибольшим весом -> добавляем ее в ind set -> берем вершину с максимальным весом
    из всех ее не соседей -> добавляем ее в ind set -> берем максимальную вершину из всех не соседей всех вершин из
    ind set -> добавляем ее в ind set -> ...
    Работает быстро (какие-то наносекунды) и в целом хорошая эвристика, показывает лучше чем 2)
    2) Эвристика поумнее, но похуже, если вкратце, то имплементировал алгоритм из Резенды, про который вы рассказывали,
    но только для взвешенного ind set. Брал вот из этой статьи: https://www.researchgate.net/publication/314654518_A_hybrid_iterated_local_search_heuristic_for_the_maximum_weight_independent_set_problem
    Работает тоже быстро (конечно в зависимости от количества итераций), но в среднем похуя
    '''
    # get ind set
    def get_ind_set(self, solution, choose_ind_set_heuristic, is_check=False):
        
        # FIRST with networkx
        # ЦИКЛ ПО ВСЕМ ВЕРШИНАМ
        if choose_ind_set_heuristic == 'first' or is_check:
            first_ind_set = []
            max_weight = -1
            free_vertices = sorted([i for i in self.vertices if solution[i - 1] not in [0, 1]], reverse=True)[:10]
            for k in free_vertices:
                ind_set = nx.maximal_independent_set(self.nx_graph, [k]) 
                ind_set_weight = sum([solution[i - 1] for i in ind_set])
                if ind_set_weight > max_weight:
                    if sorted(ind_set) not in self.save_constraints and sorted(ind_set) not in self.deleted_constraints:
                        max_weight = ind_set_weight
                        first_ind_set = ind_set
            res_ind_set = first_ind_set
                    
        # SECOND
        # ЖАДНАЯ, ВСЕГДА БЕРЕМ МАКС ВЕС
        if choose_ind_set_heuristic == 'second' or is_check:
            second_ind_set = []
            max_weight = -1
            for i in solution:
                if i > max_weight and i != 1:
                    max_weight = i
                    max_weight_ind = solution.index(max_weight) + 1
            second_ind_set.append(max_weight_ind)
            not_neighbors = list(nx.non_neighbors(self.nx_graph, max_weight_ind))
            while not_neighbors != []:
                # find max weight not neighbors
                max_weight_temp = 0
                for i in not_neighbors:
                    if solution[i - 1] >= max_weight_temp:
                        max_weight_temp = solution[i - 1]
                        max_weight_ind = i
                second_ind_set.append(max_weight_ind)
                new_not_neighbors = list(nx.non_neighbors(self.nx_graph, max_weight_ind))
                not_neighbors = list(set(not_neighbors) & set(new_not_neighbors))
            # попробуем расширить
            second_ind_set = nx.maximal_independent_set(self.nx_graph, second_ind_set)
            res_ind_set = second_ind_set
                   
        # THIRD
        # swap(w, 1)
        if choose_ind_set_heuristic == 'third' or is_check:
            third_ind_set = []
            k = 1
            for i in range(int(self.vertices_num/2)):
                free_vertices = [i for i in self.vertices if i not in third_ind_set and solution[i - 1] not in [0, 1]] # вершины, которые можем добавить
                ind_set_weight = sum([solution[i - 1] for i in third_ind_set]) # current ind_set weight
                # мы не можем в (квази)клике использовать такую эвристику
                k = np.random.choice(free_vertices) # рандомная вершина
                k_neighbors = list(nx.neighbors(self.nx_graph, k)) # соседи рандомной вершины
                k_neighbors_in_ind_set = [i for i in k_neighbors if i in third_ind_set] # какие вершины надо будет удалить
                new_vertice_weight = solution[k - 1] # вес новой вершины
                drop_weight = sum([solution[i - 1] for i in k_neighbors_in_ind_set]) # вес удаляемых вершин
                if new_vertice_weight > drop_weight:
                    third_ind_set.append(k)
                    third_ind_set = [i for i in third_ind_set if i not in k_neighbors_in_ind_set]
            # попробуем расширить
            third_ind_set = nx.maximal_independent_set(self.nx_graph, third_ind_set)
            res_ind_set = third_ind_set
    
    
    
        if is_check:
            first_weight = sum([solution[i-1] for i in first_ind_set])
            second_weight = sum([solution[i-1] for i in second_ind_set])
            third_weight = sum([solution[i-1] for i in third_ind_set])
            first_len = len(first_ind_set)
            second_len = len(second_ind_set)
            third_len = len(third_ind_set)
            
            weights = [[first_weight], [second_weight], [third_weight]]
            lens = [[first_len], [second_len], [third_len]]
            
            weights = MinMaxScaler().fit_transform(weights) + 1
            lens = MinMaxScaler().fit_transform(lens) + 1
            res_choice = list(weights * lens)
            res_choice = res_choice.index(max(res_choice)) + 1
            temp_dict = {1:'first', 2:'second', 3:'third'}
            res_ind_set = temp_dict[res_choice]

                
                
        
        return res_ind_set
    
    # add ind set constraints
    def add_ind_set_constraints(self, ind_set, solution):
        # проверяем есть ли у нас уже такое ограничение
        if sorted(ind_set) in self.save_constraints or sorted(ind_set) in self.deleted_constraints:
            self.bad_ind_set += 1
            return False
        weight_of_in_set = sum([solution[i-1] for i in ind_set])
        if weight_of_in_set < 1.35:
            self.bad_ind_set += 1
            return False
        else:
            self.added_ind_set.append(sorted(ind_set))
            self.save_constraints.append(sorted(ind_set))
            # add constraints
            constraints = []
            constraint_name = ['x{}'.format(i) for i in ind_set]
            constraints.append([constraint_name, [1] * len(ind_set)])
            constraint_senses = 'L' * len(constraints)
            right_hand_side = [1] * len(constraints)
            constraint_names = ['x{}'.format(self.ind_set_index)]
            self.model.linear_constraints.add(lin_expr=constraints,
                                                  senses=constraint_senses,
                                                  rhs=right_hand_side,
                                                  names=constraint_names)
            
            self.ind_set_index += 1
            return True
    
    
    # здесь все ветвление
    def branch_and_bounds(self):
        
        # check bad ind sets
        if self.is_ind_set == True:
            if self.bad_ind_set > self.model_start_size * (1.5 - self.graph_density):
                self.is_ind_set = False
                self.clear_model()
                self.model_size = len(self.model.linear_constraints.get_rows())
                print('STOP FININDG IND SETS')
                print('MODEL CLEARED. NEW MODEL SIZE: {}'.format(self.model_size))
        # check clear model
        model_size = len(self.model.linear_constraints.get_rows())
        if model_size >= 1.5 * self.model_size:
            self.clear_model()
            self.model_size = len(self.model.linear_constraints.get_rows())
            print('MODEL CLEARED. NEW MODEL SIZE: {}'.format(self.model_size))
        
        if self.check_time != True:
            # get current solution
            solution = self.get_solve()
            solution = [0.0  if i - 1e-9 <= 0.0 else 1.0 if i + 1e-9 >= 1.0 else i for i in solution]
    
            # Если текущее решение больше или равно уже найденной клики + 1.
            if sum(solution) >= self.current_maximum_clique + 1 - 1e-9:   
                
#                 # смотрим тип эвристики
#                 if self.choose_ind_set_heuristic == None or self.iters_check_ind_set_heuristic >= self.vertices_num * 3:
#                     if self.is_ind_set:
#                         bbvar = self.get_bb_variable(solution)
#                         if bbvar is None and sum(solution) >= self.current_maximum_clique + 1 - 1e-9:
#                             check_solution = self.is_clique(solution)
#                             if check_solution:
#                                 self.current_maximum_clique = sum(solution)
#                                 self.save_solution = solution
#                                 print('MAX CLIQUE: {}'.format(self.current_maximum_clique))
#                                 return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.constraints
#                         else:
#                             get_heuristic_type = self.get_ind_set(solution, self.choose_ind_set_heuristic, is_check=True)
#                             self.choose_ind_set_heuristic = get_heuristic_type
#                             self.iters_check_ind_set_heuristic = 0
#                             print('NEW HEURISTIC TYPE:', self.choose_ind_set_heuristic)
                
                # проверяем является ли полученное решение (квази)кликой
                bbvar = self.get_bb_variable(solution)
                if bbvar is None:
                    check_solution = self.is_clique(solution)
                    if check_solution:
                        self.current_maximum_clique = sum(solution)
                        self.save_solution = solution
                        print('MAX CLIQUE: {}'.format(self.current_maximum_clique))
                        return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.constraints
                    
                # ЗДЕСЬ НАЧИНАЕТСЯ B&C
                if self.is_ind_set:
                    prev_sum_solution = sum(solution) # сохраняем предыдущее решение чтобы посмотреть когда начинаем ветвиться
                    check_tail = 0
                    while True:
                        ind_set = self.get_ind_set(solution, self.choose_ind_set_heuristic)
                        check_ind_set_constraint = self.add_ind_set_constraints(ind_set, solution)
                        # проверяем что мы добавили новое ограничение
                        if check_ind_set_constraint == False:
                            break
                        # проверяем улучшило ли наше новое ограничение решение
                        solution = self.get_solve()
                        solution = [0.0  if i - 1e-9 <= 0.0 else 1.0 if i + 1e-9 >= 1.0 else i for i in solution] 
                        # проверяем является ли полученное решение (квази)кликой
                        bbvar = self.get_bb_variable(solution)
                        if bbvar is None:
                            check_solution = self.is_clique(solution)
                            if check_solution:
                                self.current_maximum_clique = sum(solution)
                                self.save_solution = solution
                                print('MAX CLIQUE: {}'.format(self.current_maximum_clique))
                                break
                        
                        if sum(solution) >= self.current_maximum_clique + 1 - 1e-9:
                            # проверяем является ли полученное решение (квази)кликой
                            bbvar = self.get_bb_variable(solution)
                            if bbvar is None:
                                check_solution = self.is_clique(solution)
                                if check_solution:
                                    self.current_maximum_clique = sum(solution)
                                    self.save_solution = solution
                                    print('MAX CLIQUE: {}'.format(self.current_maximum_clique))
                                    break
                            
                            
                            if sum(solution) < prev_sum_solution * 0.99:
                                prev_sum_solution = sum(solution)
                            elif check_tail >= int(self.vertices_num / 10):
                                self.bad_ind_set += 1
                                break
                            else:
                                check_tail += 1
                                if self.choose_ind_set_heuristic in ['first', 'second']:
                                    self.bad_ind_set += 1
                                    break
                        else:
                            break

            
                # ЗДЕСЬ НАЧИНАЕТСЯ B&B   
                
                # get current solution
                solution = self.get_solve()
                solution = [0.0  if i - 1e-9 <= 0.0 else 1.0 if i + 1e-9 >= 1.0 else i for i in solution]
                
                self.iters_check_ind_set_heuristic += 1
                if sum(solution) >= self.current_maximum_clique + 1 - 1e-9:
                    bbvar = self.get_bb_variable(solution)
                    # Если мы нашли integer decision
                    if bbvar is None:
                        check_solution = self.is_clique(solution)
                        if check_solution:
                            self.current_maximum_clique = sum(solution)
                            self.save_solution = solution
                            print('MAX CLIQUE: {}'.format(self.current_maximum_clique))
                            return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.constraints
                            
                    # Если решение не целочисленное
                    else:
                        self.branch_num += 1 # увеличиваем ветвление на 1
                        current_branch = self.branch_num
                        self.add_constraint_bb(bbvar, 1.0, current_branch) # добавляем ограничение
                        branch_1 = self.branch_and_bounds() # создаем первую ветку
                        self.model.linear_constraints.delete('branch_{0}'.format(current_branch)) # удаляем ее ограничения
                        self.add_constraint_bb(bbvar, 0.0, current_branch) # добавляем ограничение
                        branch_2 = self.branch_and_bounds() # создаем вторую ветку
                        self.model.linear_constraints.delete('branch_{0}'.format(current_branch)) # удаляем ее ограничения
                        return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.save_constraints, self.added_ind_set, self.bad_ind_set
                else:
                    return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.save_constraints, self.added_ind_set, self.bad_ind_set         
            else:
                return self.current_maximum_clique, self.branch_num, self.heuristic_clique, self.save_solution, self.model, self.save_constraints, self.added_ind_set, self.bad_ind_set         